In [1]:
import zipfile
import pandas as pd

In [12]:
# Nom du fichier ZIP
nom_zip = "titanic dataset.zip"

# Ouvrir le fichier ZIP
with zipfile.ZipFile(nom_zip, 'r') as zipf:
    for fichier in zipf.namelist():  # Liste des fichiers dans le ZIP
        # Ouvrir chaque fichier CSV et le lire avec pandas
        with zipf.open(fichier) as f:
            df = pd.read_csv(f)
            print("Types de données dans chaque colonne:")
            print(df.dtypes)
            print(df.head())
            #print(df.info()) 
            #print(df.describe()) 

Types de données dans chaque colonne:
PassengerId    int64
Survived       int64
dtype: object
   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1
Types de données dans chaque colonne:
PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (H

In [18]:
nb_lignes_avant = df.shape[0]
# Nombre de lignes AVANT suppression des doublons
print("Nombre de lignes avant suppression :", nb_lignes_avant)
df.duplicated().sum()
#zero double

Nombre de lignes avant suppression : 891


0

In [24]:
# Suppression des doublons
df_sans_doublons = df.drop_duplicates()

In [23]:
nb_lignes_apres = df_sans_doublons.shape[0]
# Nombre de lignes APRÈS suppression des doublons
print("Nombre de lignes après suppression :", nb_lignes_apres)

Nombre de lignes après suppression : 891


In [26]:

# Vérifier les valeurs manquantes par colonne
missing_values = df.isnull().sum()

# Afficher uniquement les colonnes qui ont des valeurs manquantes
missing_columns = missing_values[missing_values > 0]
print("Colonnes avec des valeurs manquantes :\n", missing_columns)
df_cleaned = df.dropna()  # Supprime les lignes avec des valeurs NaN

Colonnes avec des valeurs manquantes :
 Age         177
Cabin       687
Embarked      2
dtype: int64


In [27]:
print("Avant suppression :", df.shape[0])
print("Après suppression :", df_cleaned.shape[0])

Avant suppression : 891
Après suppression : 183


In [55]:
# Remplacer les valeurs NaN par la moyenne des colonnes numériques
df_filled = df.fillna(df.mean(numeric_only=True))
# avec une valeur constante adapter 
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Cabin"] = df["Cabin"].fillna("Unknown")
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0]) 

In [ ]:
#Exo3
#Créez de nouvelles fonctionnalités
df["FamilySize"] = df["SibSp"] + df["Parch"]
#extraites de la Name colonne.
df["Title"] = df["Name"].apply(lambda x: x.split(",")[1].split(".")[0].strip())
#Convertissez les variables catégorielles

In [56]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
print(categorical_columns)

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')
